## Create ML Algorithm

## Import relevant libraries

In [1]:
# we must import the libraries once again since we haven't imported them in this file
import numpy as np
import tensorflow as tf

## Import data from .npz files

We say the NPZs in 2-tuple form [inputs,targets]

In [2]:
# let's create a temporary variable npz, where we will store each of the three Audiobooks datasets
npz = np.load('Audiobooks_data_train.npz')

# we extract the inputs using the keyword under which we saved them
# to ensure that they are all floats, let's also take care of that
train_inputs = npz['inputs'].astype(np.float)
# targets must be int because of sparse_categorical_crossentropy (we want to be able to smoothly one-hot encode them)
train_targets = npz['targets'].astype(np.int)

# we load the validation data in the temporary variable
npz = np.load('Audiobooks_data_validation.npz')
# we can load the inputs and the targets in the same line
validation_inputs, validation_targets = npz['inputs'].astype(np.float), npz['targets'].astype(np.int)

# we load the test data in the temporary variable
npz = np.load('Audiobooks_data_test.npz')
# we create 2 variables that will contain the test inputs and the test targets
test_inputs, test_targets = npz['inputs'].astype(np.float), npz['targets'].astype(np.int)
train_inputs

<ipython-input-2-b1973987b5a5>:6: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  train_inputs = npz['inputs'].astype(np.float)
<ipython-input-2-b1973987b5a5>:8: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  train_targets = 

array([[-0.64419501, -0.67316726,  0.5765429 , ...,  3.45485882,
        -0.20183481,  1.79009996],
       [ 0.21053387, -0.18888517,  0.04679395, ..., -0.41569922,
        -0.20183481,  2.40415065],
       [ 0.21053387, -0.18888517, -0.18517686, ..., -0.41569922,
        -0.20183481, -0.72458383],
       ...,
       [ 0.21053387, -0.18888517, -0.39082475, ...,  3.38448504,
        -0.20183481, -0.71483699],
       [ 1.27894497,  0.41646744,  0.39886313, ...,  2.68074721,
        -0.20183481, -0.71483699],
       [-0.64419501, -0.67316726, -0.12759546, ..., -0.41569922,
        -0.20183481, -0.80255852]])

## Model

Outline, optimizers, loss, early stopping and training

*the same as the MNIST example

In [3]:
# Set the input and output sizes
input_size = 10
output_size = 2
# Use same hidden layer size for both hidden layers. Not a necessity.
hidden_layer_size = 50
    
# define how the model will look like
model = tf.keras.Sequential([
    # tf.keras.layers.Dense is basically implementing: output = activation(dot(input, weight) + bias)
    # it takes several arguments, but the most important ones for us are the hidden_layer_size and the activation function
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'), # 1st hidden layer
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'), # 2nd hidden layer
    # the final layer is no different, we just make sure to activate it with softmax
    tf.keras.layers.Dense(output_size, activation='softmax') # output layer
])


### Choose the optimizer and the loss function

# we define the optimizer we'd like to use, 
# the loss function, 
# and the metrics we are interested in obtaining at each iteration
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

### Training
# That's where we train the model we have built.

# set the batch size
batch_size = 1000

# set a maximum number of training epochs
max_epochs = 100

# set an early stopping mechanism
# let's set patience=2, to be a bit tolerant against random validation loss increases
early_stopping = tf.keras.callbacks.EarlyStopping(patience=2)

# fit the model
# note that this time the train, validation and test data are not iterable
model.fit(train_inputs, # train inputs
          train_targets, # train targets
          batch_size=batch_size, # batch size
          epochs=max_epochs, # epochs that we will train for (assuming early stopping doesn't kick in)
          # callbacks are functions called by a task when a task is completed
          # task here is to check if val_loss is increasing
          callbacks=[early_stopping], # early stopping
          validation_data=(validation_inputs, validation_targets), # validation data
          verbose = 2 # making sure we get enough information about the training process
          )  



Epoch 1/100
4/4 - 0s - loss: 0.7019 - accuracy: 0.5493 - val_loss: 0.6644 - val_accuracy: 0.6309
Epoch 2/100
4/4 - 0s - loss: 0.6527 - accuracy: 0.6275 - val_loss: 0.6268 - val_accuracy: 0.7002
Epoch 3/100
4/4 - 0s - loss: 0.6160 - accuracy: 0.6901 - val_loss: 0.5956 - val_accuracy: 0.7383
Epoch 4/100
4/4 - 0s - loss: 0.5881 - accuracy: 0.7192 - val_loss: 0.5692 - val_accuracy: 0.7539
Epoch 5/100
4/4 - 0s - loss: 0.5638 - accuracy: 0.7318 - val_loss: 0.5462 - val_accuracy: 0.7651
Epoch 6/100
4/4 - 0s - loss: 0.5427 - accuracy: 0.7485 - val_loss: 0.5256 - val_accuracy: 0.7852
Epoch 7/100
4/4 - 0s - loss: 0.5237 - accuracy: 0.7583 - val_loss: 0.5065 - val_accuracy: 0.7919
Epoch 8/100
4/4 - 0s - loss: 0.5059 - accuracy: 0.7667 - val_loss: 0.4890 - val_accuracy: 0.7919
Epoch 9/100
4/4 - 0s - loss: 0.4896 - accuracy: 0.7695 - val_loss: 0.4729 - val_accuracy: 0.7919
Epoch 10/100
4/4 - 0s - loss: 0.4746 - accuracy: 0.7734 - val_loss: 0.4583 - val_accuracy: 0.7875
Epoch 11/100
4/4 - 0s - loss:

In [4]:
train_inputs.shape[0]

3579

## Test the model

As we discussed in the lectures, after training on the training data and validating on the validation data, we test the final prediction power of our model by running it on the test dataset that the algorithm has NEVER seen before.

It is very important to realize that fiddling with the hyperparameters overfits the validation dataset. 

The test is the absolute final instance. You should not test before you are completely done with adjusting your model.

If you adjust your model after testing, you will start overfitting the test dataset, which will defeat its purpose.

In [5]:
test_loss, test_accuracy = model.evaluate(test_inputs, test_targets)

14/14 [==============================] - 0s 2ms/step - loss: 0.3556 - accuracy: 0.7924


In [6]:
print('\nTest loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))


Test loss: 0.36. Test accuracy: 79.24%


Using the initial model and hyperparameters given in this notebook, the final test accuracy should be roughly around 91%.

Note that each time the code is rerun, we get a different accuracy because each training is different. 

We have intentionally reached a suboptimal solution, so you can have space to build on it!